In [6]:
import sys
import numpy as np
import copy
from functools import lru_cache,wraps
from operator import itemgetter,countOf
import matplotlib.pyplot as plt

import collections

def read_file(file_name):
    with open(file_name) as input_file:
        list_of_lines=input_file.read().splitlines()
    i=0;
    operations=[]
    for line in list_of_lines:
        List=line.split(' ')
        
        if 'on' in List[0]:
            op=1
        else:
            op=0
        List[1]=List[1].replace('x=','')
        List[1]=List[1].replace('y=','')
        List[1]=List[1].replace('z=','')
        List[1]=List[1].split(',')
        X=List[1][0].split('..')
        Y=List[1][1].split('..')
        Z=List[1][2].split('..')
        operations.append([op,int(X[0]),int(X[1]),int(Y[0]),int(Y[1]),int(Z[0]),int(Z[1])])
    return operations
    
    
if __name__ == '__main__':
        
    Operations = read_file('input')
    Cube=np.zeros((101,101,101),np.int64)
    #print( Operations)
    for line in Operations:
        n=line[0]
        for x in range(max([-50,line[1]]),min([51,line[2]+1])):
            for y in range(max([-50,line[3]]),min([51,line[4]+1])):
                for z in range(max([-50,line[5]]),min([51,line[6]+1])):
                    Cube[x+50][y+50][z+50]=1 if n==1 else 0
    print(np.sum(Cube))
    x = range(3, 1)
    for n in x:
        print(n) 

583636


In [1]:
import sys
import numpy as np
import copy
from functools import lru_cache,wraps
from operator import itemgetter,countOf
import matplotlib.pyplot as plt

import collections

def read_file(file_name):
    with open(file_name) as input_file:
        list_of_lines=input_file.read().splitlines()
    i=0;
    operations=[]
    for line in list_of_lines:
        List=line.split(' ')
        
        if 'on' in List[0]:
            op=1
        else:
            op=0
        List[1]=List[1].replace('x=','')
        List[1]=List[1].replace('y=','')
        List[1]=List[1].replace('z=','')
        List[1]=List[1].split(',')
        X=List[1][0].split('..')
        Y=List[1][1].split('..')
        Z=List[1][2].split('..')
        operations.append([op,int(X[0]),int(X[1]),int(Y[0]),int(Y[1]),int(Z[0]),int(Z[1])])
    return operations
    
def Intersect(j,i):
    xm1,xM1,ym1,yM1,zm1,zM1=j[1:7]
    xm2,xM2,ym2,yM2,zm2,zM2=i[1:7]
    cube=[]
    #If there is an intersection
    if ( (xM2-xm1)*(xM1-xm2)>0 and
         (yM2-ym1)*(yM1-ym2)>0 and
         (zM2-zm1)*(zM1-zm2)>0 ):
        xm,xM,ym,yM,zm,zM=Get_coordinates_intersection(xm1,xM1,ym1,yM1,zm1,zM1,xm2,xM2,ym2,yM2,zm2,zM2)
        cube.append((xM-xm+1)*(yM-ym+1)*(zM-zm+1))
        cube.append(xm)
        cube.append(xM)
        cube.append(ym)
        cube.append(yM)
        cube.append(zm)
        cube.append(zM)
        return cube,True
    else:
        return cube,False

def Get_coordinates_intersection(xm1,xM1,ym1,yM1,zm1,zM1,xm2,xM2,ym2,yM2,zm2,zM2):
    xM=min([xM1,xM2])
    xm=max([xm1,xm2])
    yM=min([yM1,yM2])
    ym=max([ym1,ym2])
    zM=min([zM1,zM2])
    zm=max([zm1,zm2])
    return xm,xM,ym,yM,zm,zM

def Size_of_cube(Operations,rank):
    return abs( (Operations[rank][2]-Operations[rank][1]+1)
              * (Operations[rank][4]-Operations[rank][3]+1)
              * (Operations[rank][6]-Operations[rank][5]+1)
              )
    
# This function has found an "on" step at position rank, 
# followed only by off steps.
def Propagate(Operations,rank,N):
    #number of cubes turned on in the first step
    Nb=Size_of_cube(Operations,rank)
    if rank==N-1:
        return Nb
    #if negative steps follow
    else:
        #remove all intersecting negative cubes and compute their intersections two by two
        #them and make a list of resulting cubes
        List=[]
        for i in range(rank+1,N):
            cube,Bool=Intersect(Operations[rank],Operations[i])
            if Bool:
                print('Intersection')
                print(Operations[rank])
                print(Operations[i])
                print(cube)
                Nb -= cube[0]
                List.append(cube)
        #if two cubes or more have been removed, check for overlap
        if len(List)>1:
            #we start by adding back the cubes that have been removed twice
            Effect=1
            Nb += Iterate(List,Effect)
    return Nb
    


#@lru_cache(maxsize=1000)
def Iterate(List,Effect):
    DeltaN=0
    for cube in List:
        DeltaN += cube[0]*Effect
    #If there is more than one cube
    if(len(List)>1):
        List_Intersection=[]
        for i in range(0,len(List)-1):
            for j in range(i+1,len(List)):
                cube,Bool=Intersect(List[i],List[j])
                if Bool:
                    print('Intersection')
                    print(List[i])
                    print(List[j])
                    print(cube)
                    List_Intersection.append(cube)
        #if List non empty, continue
        if List_Intersection:  
            print(len(List_Intersection))
            print(List_Intersection)
            #we iterate with the interactions of the remaining cubes and an opposed effect
            DeltaN+=Iterate(List_Intersection,-Effect)
    return DeltaN

if __name__ == '__main__':
    
    Operations = read_file('input-test')
    N=len(Operations)
    Nb=0;
    #go through the list from the end.
    for rank in range(N-1,-1,-1):
        print('rank={} Nb={}'.format(rank,Nb))
        #if you meet an "on" step, propagate it to the end 
        #and transform it into a negative step to avoid double-counting
        if Operations[rank][0]==1:
            Nb+=Propagate(Operations,rank,N)
            Operations[rank][0]=0
    print(Nb)
    print(2758514936282235)

rank=21 Nb=0
rank=20 Nb=21049844681660
rank=19 Nb=39769201766995
rank=18 Nb=39769201894444
rank=17 Nb=39769201894444
Intersection
[1, -49, -5, -3, 45, -29, 18]
[0, -41, 9, -7, 43, -33, 15]
[78255, -41, -5, -3, 43, -29, 15]
rank=16 Nb=39769201922029
rank=15 Nb=39769201922029
Intersection
[1, -16, 35, -41, 10, -47, 6]
[0, -49, -5, -3, 45, -29, 18]
[6048, -16, -5, -3, 10, -29, 6]
Intersection
[1, -16, 35, -41, 10, -47, 6]
[0, 18, 30, -20, -8, -3, 13]
[1690, 18, 30, -20, -8, -3, 6]
Intersection
[1, -16, 35, -41, 10, -47, 6]
[0, -41, 9, -7, 43, -33, 15]
[18720, -16, 9, -7, 10, -33, 6]
Intersection
[6048, -16, -5, -3, 10, -29, 6]
[18720, -16, 9, -7, 10, -33, 6]
[6048, -16, -5, -3, 10, -29, 6]
1
[[6048, -16, -5, -3, 10, -29, 6]]
rank=14 Nb=39769202061997
rank=13 Nb=39769202061997
Intersection
[1, -18, 26, -33, 15, -7, 46]
[0, -16, 35, -41, 10, -47, 6]
[26488, -16, 26, -33, 10, -7, 6]
Intersection
[1, -18, 26, -33, 15, -7, 46]
[0, -49, -5, -3, 45, -29, 18]
[6916, -18, -5, -3, 15, -7, 18]
Inter

[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10,

[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Inter

[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Inter

Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -

[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6

Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, 

[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -

[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
3
[[2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6], [2352, -16, -5, -3, 10, -7, 6]]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6]
Intersection
[2352, -16, -5, -3, 10, -7, 6]
[2352, -16, -5, -3, 10, -7, 6

RecursionError: maximum recursion depth exceeded while calling a Python object